# MATA50 - Exercícios da Semana 03

*   Grupo: Brócolis
*   Autores: Gustavo Mendel Queiroz e Souza (Responsável), Lucas Barreto



# Instruções (não apagar)
1.   Altere o nome do notebook/documentação para o nome do vegetal do seu grupo. Por exemplo, se você é do grupo Carambola, altere o nome do notebook para "MATA50-Semana03-Carambola.ipynb"
1. Compartilhe este documento com todos os membros do grupo (para participarem da elaboração deste documento). É importante que o notebook utilizado seja o mesmo compartilhado para que os registros de participação e colaboração fiquem salvos no histórico.
1.   Incluia o nome completo dos autores na lista de autores que auxiliaram na elaboração deste notebook. Destaque o responsável como sendo o(a) primeiro(a) autor(a).
1. Utilize os recursos de blocos de texto e de código para adicionar as suas respostas, sem alterar os blocos de texto e código existente. Não economize, esses blocos são de graça.
1. Ao concluir a atividade, compartilhe o notebook com januario.ufba@gmail.com (dando permisão para edição) e deixando o aviso de notificação marcado, para que eu receba o seu e-mail.



In [ ]:
#@title Execute esta célula para carregar a classe para análise de gramáticas
class GLC:
  def __init__(self,V,Sigma,R,S):
      self.variables = V
      self.terminals = Sigma
      # self.rules = R
      self.rules = {(a, tuple(b)) for (a,b) in R}
      self.start = S
 
      self.derivation = []
      self.chomsky = None
  
  def gera(self, string):
      if (not self.chomsky):
        self.chomsky = self.to_chomsky()        
      G = self.chomsky
      
      if len(string) == 0: return (G.start,()) in G.rules
 
      # CYK Algorithm
      n = len(string)
      P = [[[] for x in range(n - y)] for y in range(n)]
      for i,c in enumerate(string):
        for v in [a for (a,b) in G.rules if len(b) == 1 and c in b]:
          P[0][i] += [v]
      for l in range(2, n+1): # borda da iteração
        for s in range(0, n+1-l): # coluna
          for p in range(1, l): # linha
            for u, v in [(a,b) for (a,b) in G.rules if len(b) == 2]:
              if v[0] in P[p-1][s] and v[1] in P[l-p-1][s+p]:
                P[l-1][s] += [u]
      return G.start in P[-1][0]
 
  def print(self):
      rightarrow = "\u2192"
      print(self.start,rightarrow, '|'.join([''.join(y) for x,y in self.rules if x == S]))
      for v in sorted(self.variables.difference(self.start)): 
        print(v,rightarrow, '|'.join([''.join(y) for x,y in self.rules if x == v]))
 
  def replace_eps(self, v, l):
    if not l: return {()}
    result = {(l[0],) + s for s in self.replace_eps(v, tuple(l[1:]))}
    if l[0] == v:
      result.update({tuple(s[1:]) for s in result})
    return result   
 
  def to_chomsky(self):
      n_state = 0
      
      S = '<{}>'.format(n_state)
      Sigma = self.terminals 
      V = {S}.union(self.variables)
      R = {(S,(self.start,))}.union(self.rules)
 
      target = {alpha for alpha in self.variables if (alpha,()) in R}
      while (target):
        for e in target:
          rules = {(alpha,beta) for (alpha,beta) in R if e in beta}
          for (a,b) in rules:
            R.update({(a, beta) for beta in self.replace_eps(e,b)})
          R.discard((e,()))
        target = {alpha for alpha in self.variables if (alpha,()) in R}
      
      target = {(a,b) for (a,b) in R if len(b) == 1 and b[0] in V}
      while (target):
        for u,v in target:
          rules = {b for (a,b) in R if a == v[0]}
          R.update({(u,b) for b in rules})
          R.discard((u,v))
        target = {(a,b) for (a,b) in R if len(b) == 1 and b[0] in V}
      
      target = {(alpha,beta) for (alpha,beta) in R if len(beta) > 2}
      while(target):
        for u,v in target:
          n_state += 1
          state = '<{}>'.format(n_state)
          V.update({state})
          R.update({(u,(v[0],state)),(state, tuple(v[1:]))})
          R.discard((u,v))
        target = {(alpha,beta) for (alpha,beta) in R if len(beta) > 2}
 
      target = {(alpha,beta) for (alpha,beta) in R if len(beta) == 2}
      for u,v in target:
        n = list(v)
        for i in range(2):
          if v[i] in Sigma:
            R.discard((u,(n[0],n[1])))
            n_state += 1
            n[i] = '<{}>'.format(n_state)
            V.update({n[i]})
            R.update({(u,(n[0],n[1]))})
            R.update({(n[i],(v[i],))})
 
      return GLC(V,Sigma,R,S)

# Exercícios

1. Construa uma gramática para a linguagem: $L = \{ w \in \{ a , b \}^* ~|~ \mbox{ o número de a's em $w$ é par} \}$. Teste a validade da sua gramática para 3 palavras de sua escolha.

In [ ]:
V     = {'P' }    # variáveis
Sigma = {'a', 'b'}          # terminais 

R     = {                   # regras
         ('P', 'PP'),
         ('P', 'aPa'),
         ('P', 'aa'),
         ('P', 'b'),
         ('P', '')         
        }     

S     = 'P'                 # partida

G = GLC(V, Sigma, R, S)
G.print()

print(G.gera("aaaa"))
print(G.gera("aba"))
print(G.gera("aaba"))

P → PP||aa|b|aPa
True
True
False


2. Construa uma gramática para a linguagem: $L = \{w \in \{a,b\}^* \mid w=w^R\}$. Teste a validade da sua gramática para 3 palavras de sua escolha.


In [ ]:
V     = {'P' }    # variáveis
Sigma = {'a', 'b'}          # terminais 

R     = {                   # regras
         ('P' , 'aPa'),
         ('P' , 'bPb'),
         ('P' , 'a'),
         ('P' , 'b'),
         ('P' , '')
        }     

S     = 'P'                 # partida

G = GLC(V, Sigma, R, S)
G.print()


print(G.gera("baab"))
print(G.gera("aaba"))
print(G.gera("ababa"))

P → |b|bPb|aPa|a
True
False
True


3. Para cada um dos problemas resolvidos URI, apresente:

*   Uma **brevíssima** descrição do problema, em suas próprias palavras.
*   A solução comentada e documentadas.
*   Utilize blocos de texto e código separados para cada atividade. 



**1º) Questão URI: 1120 - Revisão de Contrato**

*Ao registrar alguns valores de contrato, um contador percebe que houveram dígitos que não foram impressos, resultando em valores diferentes dos originais.
Assim, o programa recebe como entradas o dígito faltante e o valor original do contrato, e deve simular a impressão do contador com o dígito faltante.*

In [ ]:
''' Solução
O pyhton tem uma função bem prática para strings
Que localiza determinada sub-string e substitui ela por outra string, a função "replace"
A solução se baseia nessa função. Onde há os valores da tecla quebrada, ele substitui por uma string vazia,
Ou seja, praticamente remove o valor da string. E no final, transformo para inteiro, para remover zeros redundantes
'''

# Entrada dos dados inicial
a, b = input().split()

# Estudo dos casos de testes até receber a ordem de parada 0 e 0.
while a != "0" or b != "0":

  # Função em python para substituir strings por outra string, detro de uma string
  # Faz a substituição dos caracteres indicados por uma string vazia
  # Desse jeito, conseguimos remover da string os valores da tecla falha
	b = b.replace(a, "")
 
  # Após a remoção, se sobrar uma string vazia, é considerado o valor zero (0)
	if b == "":
		b = 0

  # Transforma a string em inteiro, para remoção de zeros redundantes  
	b = int(b)
 
  # Saída da solução
	print(b)

  # Nova entrada de dados para o próximo caso de teste
	a, b = input().split()



5 500000
0
7 7777711177777
111
0 0


**2º) Questão URI: 1234 - Sentença Dançante**

*O problema pede para converter uma sentença para o formato “AaAaAa”, chamada de “sentença dançante”. Isto é, alternando entre letras maiúsculas e minúsculas, começando sempre por uma letra maiúscula e ignorando espaços.*

In [ ]:
''' Solução
Para cada caso de teste, foi usado uma variável de controle para determinar se o caractere anterior estava Maiúsculo ou não
Passando pela string, caso o caractere não fosse um espaço, ele adicionava a uma string solução (ans) os caracteres alterados, de acordo com a variável de controle
E após isso, inverte a variável de controle, para o próximo caractere ser alternado
'''

# Pega casos de teste até EOF, caso capture a excessão de erro EOF, quebra o loop
while True:

	try:

    # Entrada de dados
		str = input()

    # Variável para solução (ANSwer)
		ans = ''

    # Variável de controle para alternância de maiúsculo e minúsculo
		control = True

    # Passear pela string através do loop, analisando os caracteres
    # Caso não for um espaço, ele vai analisar a variável de controle
    # E modificando o caractere, já adicionando na resposta final deste caso de teste
		for n in str:
			if n != ' ':
				if control:
					ans += n.upper()
				else:
					ans += n.lower()

        # Variação da variável de controle
				control = not control
			
      # Adicionando o espaço a resposta final, pois espaços ainda contam para a resposta
      # Espaços não contam apenas para a variação de Case.
			else:
				ans += n	

    # Saída da solução do caso de teste
		print(ans)

  # Caso a entrada seja um EOF, quebra o Loop e encerra os casos de teste
	except EOFError:
		break


**3º) Questão URI: 1241 - Encaixa ou não II**

*Verificar, a partir de dois valores A e B, se os dígitos de B são iguais aos últimos dígitos de A. Caso sejam, significa que B encaixa em A.*

In [ ]:
''' Solução
A solução se baseia em pegar o tamanho das strings, fazer um cálculo a partir desses valores
Pegar a sub-string da string maior de mesmo tamanho da string analisada, e verificar igualdade
'''

# Entrada da quantidade dos casos de teste
n = int(input())

# Casos de teste
while n:
	n -= 1

  # Entrada das strings a serem analisadas
	n1, n2 = input().split()

  # Pegar tamanhos das strings
	len1 = len(n1)
	len2 = len(n2)

  # Se a segunda string for maior que a primeira, ela automaticamente não encaixa
	if len2 > len1:
		print("nao encaixa")
		continue

  # Variável auxiliar para pegar a sub-string final a ser analisada
	aux = ''

  # Loop entre a primeira string para analisar apenas os últimos caracteres
  # Adicionando à variável auxiliar
	for i in range(len1-len2, len1):
		aux += n1[i]

  # Verificação de igualdade e resultado final do caso de teste
	if aux == n2:
		print("encaixa")
	else:
		print("nao encaixa")
